In [1]:
import os
import math
import statistics
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import skimage.feature
from skimage import feature
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb



In [2]:
path="TD4-database/train/"
folders=os.listdir(path)
print(folders)

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [3]:
#lets create a local binary pattern class

class LBP:
  def __init__(self, P, R, image):
    self.P = P
    self.R = R
    self.image = image

  def get_histogram(self):
    # Create the Local Binary Pattern object and get the histogram of it
    lbp = feature.local_binary_pattern(self.image, self.P, self.R, method="uniform")
    (histogram, _) = np.histogram(lbp.ravel(), bins=np.arange(0, self.P + 3), range=(0, self.P + 2))
    histogram = histogram.astype("float")
    #histogram /= (histogram.sum() + 0.000001) # to avoid devision by zero add a small value (0.000001)
    
    return histogram

In [4]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)


R = 4
P = 8*R

lable_list_train=[] # save the emotion lable  [0 'angry', 1'disgust', 2'fear', 3'happy', 4'neutral', 5'sad', 6'surprise']
imgs_train=[] #save the imamges
for i in range(len(folders)):
    path_emotion=path+folders[i]
    filenames=sorted(os.listdir(path_emotion))      
    for j in range(len(filenames)):
        lable_list_train.append(i)
        img = plt.imread(path_emotion+"/"+filenames[j])
        histogram = LBP(P, R, img).get_histogram()
        imgs_train.append(histogram)
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 22:25:39
Current Time = 22:27:15


In [5]:
path_test="TD4-database/test/"
folders_test=os.listdir(path_test)
print(folders_test)

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [6]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

lable_list_test=[] # save the emotion lable  [0 'angry', 1'disgust', 2'fear', 3'happy', 4'neutral', 5'sad', 6'surprise']
imgs_test=[] #save the imamges

for i in range(len(folders_test)):
    path_emotion=path_test+folders_test[i]
    filenames=sorted(os.listdir(path_emotion))      
    for j in range(len(filenames)):
        lable_list_test.append(i)
        img_=plt.imread(path_emotion+"/"+filenames[j])
        histogram = LBP(P, R, img_).get_histogram()
        imgs_test.append(histogram)

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 22:27:15
Current Time = 22:27:37


In [7]:
# convert list to numpy array
imgs_train = np.array(imgs_train, dtype=np.float32)
imgs_test = np.array(imgs_test, dtype=np.float32)

lable_list_train = np.array(lable_list_train, dtype=np.float32)
lable_list_test = np.array(lable_list_test, dtype=np.float32)

In [8]:
np.save('imgs_train.npy', imgs_train) 
np.save('imgs_test.npy', imgs_test) 

np.save('lable_list_train.npy', lable_list_train) 
np.save('lable_list_test.npy', lable_list_test) 

In [9]:
imgs_train = np.load('imgs_train.npy')
imgs_test = np.load('imgs_test.npy')

lable_list_train = np.load('lable_list_train.npy')
lable_list_test = np.load('lable_list_test.npy')

In [10]:
imgs_train

array([[ 121.,   62.,   37., ...,   45.,   53., 1028.],
       [ 119.,   36.,   44., ...,   35.,   64., 1119.],
       [   0.,    0.,    0., ...,    0., 2304.,    0.],
       ...,
       [ 125.,   54.,   36., ...,   31.,   79.,  963.],
       [ 140.,   65.,   51., ...,   39.,   60., 1194.],
       [ 117.,   54.,   66., ...,   27.,   70., 1201.]], dtype=float32)

In [11]:
imgs_test

array([[  97.,   52.,   59., ...,   37.,   61., 1021.],
       [ 128.,   50.,   43., ...,   30.,   50., 1085.],
       [  93.,   25.,   40., ...,   21.,   51.,  581.],
       ...,
       [ 130.,   60.,   70., ...,   10.,   59., 1062.],
       [  89.,   45.,   31., ...,   15.,   48.,  827.],
       [ 111.,   47.,   39., ...,   21.,   70., 1120.]], dtype=float32)

In [12]:
# # lets use KNN neighbour classifier with 3 neighors
# neigh = KNeighborsClassifier(n_neighbors=3)
# # lets fit the model
# neigh.fit(imgs, lable_list) 

In [ ]:
def Grid_Search_CV_KNN(imgs, lable_list):
    estimator = KNeighborsClassifier()

    parameters = {'n_neighbors': [10, 20, 30, 40, 50],
              'weights': ['uniform', 'distance'],
               'algorithm': ['auto', 'ball_tree'],
              'p': [1, 2],
               }

    grid = GridSearchCV(estimator, parameters, n_jobs=-1, cv=5)

    grid.fit(imgs, lable_list)

    return grid.best_score_ , grid.best_params_

def KNN(imgs_train, imgs_test, lable_list_train, lable_list_test, best_params):
    estimator = KNeighborsClassifier(n_jobs=-1).set_params(**best_params)
    estimator.fit(imgs_train,lable_list_train)
    y_predict = estimator.predict(imgs_test)
    return lable_list_test,y_predict

RMSE = []
R2 = []
BEST_SCORE = []
for i in range(2):
    print ("iteration: " , i)
    print ("---------------")
    best_score, best_params = Grid_Search_CV_KNN(imgs_train, lable_list_train)
    lable_list_test,y_predict = KNN(imgs_train, imgs_test, lable_list_train, lable_list_test, best_params)
    rmse = math.sqrt(np.mean((y_predict - lable_list_test) ** 2))
    r2 = r2_score(lable_list_test,y_predict)
    RMSE.append(round(rmse, 5))
    R2.append(round(r2, 5))
    BEST_SCORE.append(round(best_score, 5))

    print ("Best Score:" ,best_score)
    print ("Best params:",best_params)

print("Average RMSE: ", statistics.mean(RMSE))
print("Average R2: ", statistics.mean(R2))
print("Average BEST_SCORE: ", statistics.mean(BEST_SCORE))

iteration:  0
---------------
Best Score: 0.32571688086166517
Best params: {'algorithm': 'auto', 'n_neighbors': 50, 'p': 1, 'weights': 'distance'}
iteration:  1
---------------


In [ ]:
#
#def Grid_Search_CV_XGB(X_train, y_train):
#    estimator = xgb.XGBClassifier()
#
#    parameters = {'n_estimators': [200],
#              'learning_rate': [0.1],
#               'gamma': [0],
#              'max_depth': [7],
#              'subsample': [0.75],
#              'colsample_bytree': [1],
#              'booster': ['gbtree', 'dart'],
#             }
#
#    grid = GridSearchCV(estimator, parameters, n_jobs=-1, cv=40)
#
#    grid.fit(X_train, y_train)
#
#    return grid.best_score_ , grid.best_params_
#
#def XGB(X_train, X_test, y_train, y_test, best_params):
#    estimator = xgb.XGBClassifier(n_jobs=-1).set_params(**best_params)
#    estimator.fit(X_train,y_train)
#    y_predict = estimator.predict(X_test)
#    return y_test,y_predict
#
#
#RMSE = []
#R2 = []
#BEST_SCORE = []
#for i in range(2):
#    print ("iteration: " , i)
#    print ("---------------")
#    best_score, best_params = Grid_Search_CV_XGB(imgs_train, lable_list_train)
#    lable_list_test,y_predict = XGB(imgs_train, imgs_test, lable_list_train, lable_list_test, best_params)
#    rmse = math.sqrt(np.mean((y_predict - lable_list_test) ** 2))
#    r2 = r2_score(lable_list_test,y_predict)
#    RMSE.append(round(rmse, 5))
#    R2.append(round(r2, 5))
#    BEST_SCORE.append(round(best_score, 5))
#
#    print ("Best Score:" ,best_score)
#    print ("Best params:",best_params)
#
#print("Average RMSE: ", statistics.mean(RMSE))
#print("Average R2: ", statistics.mean(R2))
#print("Average BEST_SCORE: ", statistics.mean(BEST_SCORE))
#
#

In [ ]:
#Test Random Forest
def Grid_Search_CV_RFR(imgs, lable_list):

    estimator = RandomForestClassifier()
    param_grid = { 
            "n_estimators"      : [50,100],
            "criterion"         : ['gini', 'entropy'],
            "max_features"      : ["sqrt", 'auto'],
            "min_samples_split" : [8],
            "bootstrap"         : [False, True],
            }

    grid = GridSearchCV(estimator, param_grid, n_jobs=-1, cv=10)

    grid.fit(imgs, lable_list)

    return grid.best_score_ , grid.best_params_

def RFR(imgs_train, imgs_test, lable_list_train, lable_list_test, best_params):
    estimator = RandomForestClassifier(n_jobs=-1).set_params(**best_params)
    estimator.fit(imgs_train,lable_list_train)
    y_predict = estimator.predict(imgs_test)
    return lable_list_test,y_predict

RMSE = []
R2 = []
BEST_SCORE = []
for i in range(2):
    print ("iteration: " , i)
    print ("---------------")
    best_score, best_params = Grid_Search_CV_RFR(imgs_train, lable_list_train)
    lable_list_test,y_predict = RFR(imgs_train, imgs_test, lable_list_train, lable_list_test, best_params)
    rmse = math.sqrt(np.mean((y_predict - lable_list_test) ** 2))
    r2 = r2_score(lable_list_test,y_predict)
    RMSE.append(round(rmse, 5))
    R2.append(round(r2, 5))
    BEST_SCORE.append(round(best_score, 5))

    print ("Best Score:" ,best_score)
    print ("Best params:",best_params)

print("Average RMSE: ", statistics.mean(RMSE))
print("Average R2: ", statistics.mean(R2))
print("Average BEST_SCORE: ", statistics.mean(BEST_SCORE))